In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterSampler
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import re

# sklearn classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

%load_ext autoreload
%autoreload 2

Need a class that creates all the parameters for the classifier and random search. Needs to be reproducible.

Random search parameters:

* random seed
* number of iterations
* under/over sampling method
* imbalance ratio (for under/over sampling)
* scaling method (e.g. standard scaling, or min/max, etc.)
* features used
    * how to repeatably select the appropriate features? Maybe randomly shuffle?
    * use `random.choices` to select the features? Use `random.seed()` to set the seed.
* number of features used
* classifier used
* data set used

Dataset parameters (may be specific for dataset). Should probably store as a separate data card... For milling data set, parameters could be:

* label meta-data (df_labels)
* window size
* stride
* cut drop list

Classifier parameters:
* classifier
* parameter sampler seed


In [26]:
# practice loading the datafram, selecting the column names, and randomly selecting n number of features (based on column names)

def build_classifier_parameter_dict_named(clf, p_clf):

    name = re.sub("'", "", str(type(clf)).replace(">", "").split(".")[-1])

    # rebuild the parameter dict and append the classifier name onto each parameter
    classifier_parameters = {}
    for k in p_clf:
        classifier_parameters[str(name) + "_" + k] = p_clf[k]

    return classifier_parameters


def random_forest_classifier(sampler_seed, parameters_sample_dict=None):
    if parameters_sample_dict is None:
        parameters_sample_dict = {
            "n_estimators": sp_randint(5, 500),
            "max_depth": sp_randint(1, 500),
            "random_state": sp_randint(1, 2 ** 16),
            "min_samples_leaf": sp_randint(1, 10),
            "bootstrap": [True, False],
            "class_weight": ['balanced','balanced_subsample',None],
        }

    param_dict_raw = list(
        ParameterSampler(
            parameters_sample_dict, n_iter=1, random_state=sampler_seed
        )
    )[0]

    clf = RandomForestClassifier(**p_clf)

    return clf, param_dict_raw, build_classifier_parameter_dict_named(clf, p_clf)

In [27]:
clf, p_clf, param_dict_named = random_forest_classifier(1)
param_dict

{'RandomForestClassifier_bootstrap': False,
 'RandomForestClassifier_class_weight': 'balanced',
 'RandomForestClassifier_max_depth': 73,
 'RandomForestClassifier_min_samples_leaf': 6,
 'RandomForestClassifier_n_estimators': 340,
 'RandomForestClassifier_random_state': 21441}

In [28]:
p_clf

{'bootstrap': False,
 'class_weight': 'balanced',
 'max_depth': 73,
 'min_samples_leaf': 6,
 'n_estimators': 340,
 'random_state': 21441}